In [1]:
from pathlib import Path
from enum import StrEnum


class STAGES(StrEnum):
    RIGID = "Rigid"
    AFFINE = "Affine"
    SYN = "SyN"


class EXPERIMENTS(StrEnum):
    AMP = "amp"
    FP64 = "fp64"


N_LEVELS = 4


BOKEH_COLORBLIND = (
    "#0072B2",  # 0
    "#E69F00",  # 1
    "#F0E442",  # 2
    "#009E73",  # 3
    "#56B4E9",  # 4
    "#D55E00",  # 5
    "#CC79A7",  # 6
    "#000000",  # 7
)

# MODIFY IF NEEDED
FIG_DIR = Path("paper", "figures")
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Data ingest

In [2]:
from collections import defaultdict
import re
from typing import Any, Callable

import pandas as pd


def cleanup_level(levels: list[str]) -> list[str]:
    rv: list[str] = []
    for level in levels:
        clean_level = level.strip("\n").strip()
        if not (
            clean_level.startswith("2DIAGNOSTIC")
            or clean_level.startswith("1DIAGNOSTIC")
        ):
            continue
        rv.append(clean_level)
    return rv


def data2table_amp(data: str) -> pd.DataFrame:
    table: defaultdict[str, Any] = defaultdict(list)
    for line in data.splitlines():
        if line in ["", "XX"]:
            continue
        cols = line.split(",")
        table["iteration"].append(int(cols[1]))
        table["metricValue"].append(float(cols[2]))
        table["vprec_precision"].append(int(cols[6]))
        table["pmin_estimate"].append(float(cols[7]))
        table["pmin_estimate_avg"].append(float(cols[8]))

    return pd.DataFrame(table | {"exp_id": EXPERIMENTS.AMP})


def data2table_fp64(data: str) -> pd.DataFrame:
    table: defaultdict[str, Any] = defaultdict(list)
    for line in data.splitlines():
        if line in ["", "XX"]:
            continue
        cols = line.split(",")
        table["iteration"].append(int(cols[1]))
        table["metricValue"].append(float(cols[2]))

    return pd.DataFrame(table | {"exp_id": EXPERIMENTS.FP64})


def parse_log(
    txt: str, *, data2table_fn: Callable[[str], pd.DataFrame]
) -> pd.DataFrame:
    P_LEVEL_HEADER = r"DIAGNOSTIC,Iteration,metricValue,convergenceValue,ITERATION_TIME_INDEX,SINCE_LAST.*|  Elapsed time"
    levels = re.split(P_LEVEL_HEADER, txt)
    raw_data = cleanup_level(levels)

    parsed_levels: list[pd.DataFrame] = []
    for i, stage in enumerate(STAGES):
        for level in range(N_LEVELS):
            level_data = raw_data[level + i * N_LEVELS]
            table = data2table_fn(level_data)
            table["stage"] = stage
            table["level"] = level
            parsed_levels.append(table)

    return pd.concat(parsed_levels)


def get_data(
    log_dir: Path, *, data2table_fn: Callable[[str], pd.DataFrame]
) -> pd.DataFrame:
    data: list[pd.DataFrame] = list()

    for log in log_dir.glob("*.log"):
        subject_df = parse_log(
            log.read_text(),
            data2table_fn=data2table_fn,
        )
        subject_df["subject_id"] = log.stem
        data.append(subject_df)

    return pd.concat(data).reset_index(drop=True)


amp_df = get_data(Path("logs", EXPERIMENTS.AMP), data2table_fn=data2table_amp)
fp64_df = get_data(Path("logs", EXPERIMENTS.FP64), data2table_fn=data2table_fp64)

all_df: pd.DataFrame = pd.concat([amp_df, fp64_df]).reset_index(drop=True)
all_df

,iteration,metricValue,vprec_precision,pmin_estimate,pmin_estimate_avg,exp_id,stage,level,subject_id
0,1,-0.567814,8.0,2.982600,2.982600,amp,Rigid,0,0003059
1,2,-0.568658,8.0,3.617058,3.299829,amp,Rigid,0,0003059
2,3,-0.570341,8.0,3.949972,3.516543,amp,Rigid,0,0003059
3,4,-0.572271,8.0,4.260256,3.702472,amp,Rigid,0,0003059
4,5,-0.575280,8.0,3.620636,3.686104,amp,Rigid,0,0003059
...,...,...,...,...,...,...,...,...,...
42567,16,-0.896145,NaN,NaN,NaN,fp64,SyN,3,0003040
42568,17,-0.896713,NaN,NaN,NaN,fp64,SyN,3,0003040
42569,18,-0.897232,NaN,NaN,NaN,fp64,SyN,3,0003040
42570,19,-0.897713,NaN,NaN,NaN,fp64,SyN,3,0003040


# Plotting utilities

In [3]:
import plotly.graph_objects as go
from plotly.colors import hex_to_rgb
from plotly.subplots import make_subplots


def hex_to_rgba(hex_color: str, alpha: float) -> str:
    r, g, b = hex_to_rgb(hex_color)
    return f"rgba({r}, {g}, {b}, {alpha})"


def adjust_layout(fig: go.Figure, title: str):
    fig.update_layout(
        height=len(STAGES) * 400,
        width=1200,
        font=dict(
            size=24,
            color="black",
        ),
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.025,  # position above the subplots
            xanchor="left",
            x=0,
            traceorder="normal",
        ),
        title=title,
        margin=dict(l=10, r=40, t=150, b=10),
        plot_bgcolor="white",
    )
    for i in range(1, len(STAGES) * N_LEVELS + 1):
        fig.update_layout(
            {
                f"xaxis{i}": dict(showline=True, linewidth=2, linecolor="black"),
                f"yaxis{i}": dict(showline=True, linewidth=2, linecolor="black"),
            }
        )
    fig.update_yaxes(rangemode="tozero")


def get_base_fig(title: str) -> go.Figure:
    fig = make_subplots(
        rows=len(STAGES),
        cols=N_LEVELS,
        subplot_titles=[
            f"{stage}: Level {level}"
            for _, stage in enumerate(STAGES)
            for level in range(N_LEVELS)
        ],
        shared_xaxes=False,
        shared_yaxes=True,
        horizontal_spacing=0.025,
        vertical_spacing=0.075,
    )
    # Update subplot titles font size
    for annotation in fig.layout.annotations:
        annotation.font = dict(size=18)

    adjust_layout(fig, title=title)

    return fig


def show_and_save(fig: go.Figure, out_file: Path | None, show: bool):
    if show:
        fig.show()
    if out_file is not None:
        fig.write_image(str(out_file), scale=2)


# Precision requirement per iteration


In [4]:
HLINE_COLOR = "darkgray"


class COLOR(StrEnum):
    VPREC_PRECISION = BOKEH_COLORBLIND[0]
    PMIN_ESTIMATE = BOKEH_COLORBLIND[1]


def add_hline(fig: go.Figure, y: int, text: str) -> None:
    for i, stage in enumerate(STAGES, 1):
        if stage == STAGES.SYN and y >= 24:
            return
        # Only show the horizontal line in the last column
        line = dict(dash="solid", width=1, color=HLINE_COLOR)
        fig.add_hline(
            y,
            row=i,
            line=line,
        )
        fig.add_hline(
            y,
            row=i,
            col=N_LEVELS,
            line=line,
            annotation_text=text,
            annotation_position="right",
            annotation_xref="paper",
            annotation_font_size=12,
            annotation_showarrow=False,
            annotation_font_color="black",
        )


def display_hardware_support(fig: go.Figure, hw_support: dict[str, int]):
    for k, v in hw_support.items():
        add_hline(fig, y=v, text=k)


def _trace_boudary(
    fig: go.Figure,
    *,
    x: pd.Series,
    upper: pd.Series,
    lower: pd.Series,
    mean: pd.Series,
    row: int,
    col: int,
    color: str,
    dash: str,
):
    # Add the Upper Bound (Max)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=upper,
            line=dict(width=0),
            showlegend=False,
        ),
        row=row,
        col=col,
    )

    # Add lower bound (Min)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=lower,
            line=dict(width=0),
            fill="tonexty",  # This performs the shading
            fillcolor=hex_to_rgba(color, 0.2),  # Gray with 20% opacity
            showlegend=False,
        ),
        row=row,
        col=col,
    )

    # Mean line
    fig.add_trace(
        go.Scatter(
            x=x,
            y=mean,
            line=dict(
                color=color,
                width=3,
                dash=dash,
            ),
            showlegend=False,
        ),
        row=row,
        col=col,
    )


def trace_minmax(
    fig: go.Figure,
    stats: dict[str, pd.DataFrame],
    row: int,
    col: int,
    color: str,
    dash: str = "solid",
) -> None:
    _trace_boudary(
        fig,
        x=stats.index,
        upper=stats["max"],
        lower=stats["min"],
        mean=stats["mean"],
        row=row,
        col=col,
        color=color,
        dash=dash,
    )


def trace_std(
    fig: go.Figure,
    stats: dict[str, pd.DataFrame],
    row: int,
    col: int,
    color: str,
    dash: str = "solid",
) -> None:
    _trace_boudary(
        fig,
        x=stats.index,
        upper=stats["mean"] + stats["std"],
        lower=stats["mean"] - stats["std"],
        mean=stats["mean"],
        row=row,
        col=col,
        color=color,
        dash=dash,
    )


def make_legend(fig: go.Figure):
    fig.add_trace(
        go.Scatter(
            x=[1],
            y=[0],
            name="Virtual precision",
            line=dict(dash="solid", color=COLOR.VPREC_PRECISION),
            mode="lines",
            showlegend=True,
            hoverinfo="skip",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=[1],
            y=[0],
            name="Estimated pmin",
            line=dict(dash="dash", color=COLOR.PMIN_ESTIMATE),
            mode="lines",
            showlegend=True,
            hoverinfo="skip",
        ),
        row=1,
        col=1,
    )
    # Only add this ONCE (e.g., at subplot [1,1] or outside the loop)
    fig.add_trace(
        go.Scatter(
            x=[1],
            y=[0],
            mode="lines",
            line=dict(dash="solid", color=HLINE_COLOR),
            name="Hardware support",
            showlegend=True,
            hoverinfo="skip",
        ),
        row=1,
        col=1,
    )


def plot_pmin(
    df: pd.DataFrame,
    *,
    trace_fn,
    title: str,
    out_file: Path | None = None,
    show: bool = True,
) -> None:
    fig = get_base_fig(title)

    for i, stage in enumerate(STAGES):
        for level in range(N_LEVELS):
            stage_level_df = df[(df["stage"] == stage) & (df["level"] == level)]

            # Virtual precision
            stats = (
                stage_level_df.groupby("iteration")["vprec_precision"]
                .agg(["mean", "min", "max", "std"])
                .reset_index()
            )
            trace_fn(
                fig,
                stats,
                row=i + 1,
                col=level + 1,
                dash="solid",
                color=COLOR.VPREC_PRECISION,
            )
            # Estimated pmin
            stats = (
                stage_level_df.groupby("iteration")["pmin_estimate_avg"]
                .agg(["mean", "min", "max", "std"])
                .reset_index()
            )
            trace_fn(
                fig,
                stats,
                row=i + 1,
                col=level + 1,
                dash="dot",
                color=COLOR.PMIN_ESTIMATE,
            )

    make_legend(fig)
    hw_support = {"FP32": 24, "FP24": 16, "TF32": 11, "BF16": 8}
    hw_support = {"FP64": 53, "FP32": 24, "FP24": 16, "TF32": 11, "BF16": 8}
    display_hardware_support(fig, hw_support=hw_support)

    # x-axis title
    fig.add_annotation(
        text="nth Iteration",
        xref="paper",
        yref="paper",
        x=0.5,
        y=-0.075,
        showarrow=False,
        font=dict(size=24),
    )
    fig.update_layout(margin=dict(b=80))

    show_and_save(fig, out_file=out_file, show=show)

In [5]:
plot_data = all_df[all_df["exp_id"] == EXPERIMENTS.AMP]
plot_pmin(
    plot_data,
    trace_fn=trace_minmax,
    title="Pmin: MinMax bounds",
    out_file=FIG_DIR / "pmin_minmax_bounds.png",
)
plot_pmin(
    plot_data,
    trace_fn=trace_std,
    title="Pmin: Std bounds",
    out_file=FIG_DIR / "pmin_std_bounds.png",
)

# Relative difference (AMP vs. FP64)


In [6]:
df_last = all_df.sort_values("iteration", ascending=False).drop_duplicates(
    subset=["subject_id", "stage", "level", "exp_id"], keep="first"
)

df_pivot = df_last.pivot_table(
    index=["subject_id", "stage", "level"],
    columns="exp_id",
    values="metricValue",
).reset_index()

df_pivot["diff_fp64_amp"] = (df_pivot["amp"] - df_pivot["fp64"]) / df_pivot[
    "fp64"
].abs()
df_pivot


exp_id,subject_id,stage,level,amp,fp64,diff_fp64_amp
0,0003001,Affine,0,-0.871946,-0.871005,-1.080184e-03
1,0003001,Affine,1,-0.730982,-0.730976,-8.509114e-06
2,0003001,Affine,2,-0.620231,-0.620194,-5.956523e-05
3,0003001,Affine,3,-0.551107,-0.551397,5.243211e-04
4,0003001,Rigid,0,-0.665033,-0.665033,6.037296e-10
...,...,...,...,...,...,...
595,0003064,Rigid,3,-0.351596,-0.351596,0.000000e+00
596,0003064,SyN,0,-0.973531,-0.973531,4.917665e-09
597,0003064,SyN,1,-0.946442,-0.946442,-8.424920e-09
598,0003064,SyN,2,-0.932349,-0.932349,4.901598e-11


In [7]:
def plot_rel_diff(
    df: pd.DataFrame,
    *,
    value_col: str,
    title: str,
    out_file: Path | None = None,
    show: bool = True,
) -> None:
    fig = get_base_fig(title)

    for i, stage in enumerate(STAGES):
        for level in range(N_LEVELS):
            stage_level_df = df[(df["stage"] == stage) & (df["level"] == level)]

            fig.add_trace(
                go.Box(
                    y=stage_level_df[value_col],
                    boxpoints="all",
                    jitter=0.5,
                    pointpos=0,
                    marker=dict(color=hex_to_rgba(BOKEH_COLORBLIND[0], 0.4), size=8),
                    line=dict(color=BOKEH_COLORBLIND[7]),
                    showlegend=False,
                    name="",
                ),
                row=i + 1,
                col=level + 1,
            )

    for j in range(1, len(STAGES) * N_LEVELS + 1):
        fig.update_layout({f"yaxis{j}": dict(tickformat=".1e", nticks=5)})

    show_and_save(fig, out_file=out_file, show=show)

In [8]:
plot_rel_diff(
    df_pivot,
    value_col="diff_fp64_amp",
    title="Relative Difference in Metric Value (FP64 - AMP)",
    out_file=FIG_DIR / "rel_diff_fp64_amp.png",
)